In [27]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))   

/var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/ipykernel_55529/1194679256.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [25]:
import librosa
import numpy as np
import soundfile as sf
import os
import glob
from tqdm import tqdm

In [39]:
def change_volume(y, factor):
    return y * factor

def add_noise(y, noise_factor=0.005):
    noise = np.random.randn(len(y))
    return y + noise_factor * noise

def remove_parts(y, sr, remove_ratio=0.1):
    y_len = len(y)
    part_len = int(y_len * remove_ratio)
    start = np.random.randint(0, y_len - part_len)
    y[start:start + part_len] = 0
    return y

def change_speed(y, speed_factor):
    return librosa.effects.time_stretch(y, rate=speed_factor)

def add_silence(y, sr, position='beginning', silence_duration_ms=100):
    silence = np.zeros(int(sr * silence_duration_ms / 1000))
    if position == 'beginning':
        return np.concatenate([silence, y])
    elif position == 'end':
        return np.concatenate([y, silence])

def save_audio(y, sr, filename):
    sf.write(filename, y, sr)

def create_variations(audio_path):
    #print(f"Processing {audio_path}...")
    y, sr = librosa.load(audio_path, sr=None)
    
    base_filename, ext = os.path.splitext(audio_path)
    base_filename = os.path.join(os.path.dirname(base_filename), os.path.basename(base_filename).replace(' ', '_'))

    # Higher volume
    y_high_vol = change_volume(y, 1.5)
    save_audio(y_high_vol, sr, f'{base_filename}_high_volume{ext}')

    # Lower volume
    y_low_vol = change_volume(y, 0.5)
    save_audio(y_low_vol, sr, f'{base_filename}_low_volume{ext}')

    # Added noise
    y_noise = add_noise(y)
    save_audio(y_noise, sr, f'{base_filename}_noise{ext}')

    # Parts missing
    y_missing = remove_parts(y.copy(), sr)
    save_audio(y_missing, sr, f'{base_filename}_parts_missing{ext}')

    # Faster
    y_faster = change_speed(y, 1.25)
    save_audio(y_faster, sr, f'{base_filename}_faster{ext}')

    # Slower
    y_slower = change_speed(y, 0.75)
    save_audio(y_slower, sr, f'{base_filename}_slower{ext}')

    # 100ms of silence at the beginning
    y_silence_begin = add_silence(y, sr, position='beginning', silence_duration_ms=100)
    save_audio(y_silence_begin, sr, f'{base_filename}_silence_begin{ext}')

    # 100ms of silence at the end
    y_silence_end = add_silence(y, sr, position='end', silence_duration_ms=100)
    save_audio(y_silence_end, sr, f'{base_filename}_silence_end{ext}')
    
    #print(f"Finished processing {audio_path}")

def process_directory(directory_path):
    mp3_files = glob.glob(os.path.join(directory_path, '*.wav'))
    print(f"Found {len(mp3_files)} files.")
    
    for mp3_file in tqdm(mp3_files, desc="Processing files"):
        create_variations(mp3_file)

directory_path = '/Users/ciprian/Desktop/Projects/Smart Plant Pot/Audio/Voice Recognition/Prototype 3/noise'
process_directory(directory_path)

Found 1000 files.


Processing files: 100%|█████████████████████| 1000/1000 [00:32<00:00, 31.15it/s]
